In [ ]:
import numpy as np
from numpy import float64 as realN

from intervalN import *
from reachN import *

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from DaTaReachControl import generateTraj, synthNextState, synthTraj

In [ ]:
##### Define the Unicycle system dynamics and the initial trajectory is generated ###
#####################################################################################

# Define a seed for repeatability
np.random.seed(2339) # 3626 923 sexy traj: (3963 c_vmax =  1.0 c_wmax =  1.5 c_rot = 5)
# val = int(np.random.uniform(0,4000))
# print (val)
# np.random.seed(val)

# Sampling time
sampling_time = 0.1

# Define the initial satte and the axis limits
initial_state = np.array([-2, -2.5, np.pi/2])

# Number of data in initial trajectory
n_data_max = 15

# Input bounds
v_max = 4
w_max = 0.5 * (2*np.pi)
v_min = -v_max
w_min = -w_max
input_lb = np.array([v_min, w_min])
input_ub = np.array([v_max, w_max])

# Generate random input sequence --> put some random zero
# in some direction of the control to have a trajectory with certain quality
v_seq = -1 *(np.random.rand(n_data_max,1) - 0) * v_max # 2 * (np.random.rand(n_data_max,1) - 0.5) * v_max
w_seq = 2 * (np.random.rand(n_data_max,1) - 0.5) * w_max
# Number of data points with potentially zero for the value of the control in a direction
nSep = int(n_data_max /2)
# Randomly pick the indexes with zero control in a random direction
sepIndexes = np.random.choice([ i for i in range(n_data_max)], nSep, replace=False)
for i in range(1,nSep):
    zero_or_no = np.random.choice([0,1], p=[0.2,0.8])
    if zero_or_no == 0:
        w_seq[sepIndexes[i],0] = 0
        v_seq[sepIndexes[i],0] = 0
        continue
    v_or_theta = np.random.choice([0,1], p=[0.4,0.6])
    if v_or_theta == 0: # pick v
        w_seq[sepIndexes[i],0] = 0
    else: # pick theta
        v_seq[sepIndexes[i],0] = 0
rand_control = np.hstack((v_seq,w_seq))
# print (rand_init_input_vec)

# Input signal to evaluate a trajectory of the system
c_vmax =  1.0
c_wmax =  1.
c_rot = 6
t_end = (n_data_max-1) * sampling_time

# Build the uncertain control (here, there is no uncertaincy)
@jit(nopython=True, parallel=False, fastmath=True)
def uOver(t_lb, t_ub):
    if t_lb <= t_end+(0.9*sampling_time):
        indx = int(t_lb / sampling_time)
        return rand_control[indx,:], rand_control[indx,:] 
    x_lb = np.empty(2, dtype=realN)
    x_ub = np.empty(2, dtype=realN)
    x_lb[0] = c_vmax
    x_ub[0] = c_vmax
    x_lb[1], x_ub[1] = cos_i(c_rot*(t_lb-t_end), c_rot*(t_ub-t_end))
    x_lb[1], x_ub[1] = mul_i_scalar(x_lb[1], x_ub[1], c_wmax)
    return x_lb, x_ub

# Unknown dynamics of the unicycle
def fFun(x):
    return np.zeros(3, dtype=np.float64)
def GFun(x):
    return np.array([[np.cos(x[2]), 0], [np.sin(x[2]), 0], [0.0, 1.0]])
# Use the uncertain control above as the control signal to generate a trajectory
def uFun(t):
    u_t, _ = uOver(t, t)
    return u_t.reshape(-1,1)

# Generate a more precise trajectory with smaller sample time based on the control
# ufun by solving numerically the ODE with the true dynamics
nPoint = 81
dt = 0.05 # Delta time of the computed trajectory

# Create the time tables
tVal_1 = np.array([ i*dt for i in range(nPoint+1)])
tMeas = np.array([i*sampling_time for i in range(n_data_max)])
# Obtain the trajectory via ode solver
tVal_1, traj, trajDot = synthTraj(fFun, GFun, uFun, initial_state, tVal_1, atol=1e-10, rtol=1e-10)
tVal = tVal_1[:-1] # Obtain the list of time before the last time
# Create a table with the control values applied at each time step dt
u_values = np.empty((rand_control.shape[1], nPoint), dtype=np.float64)
for i in range(u_values.shape[1]):
    t_lb, t_ub = uOver(tVal[i], tVal[i])
    u_values[:,i] = t_lb

# Construct the state evolution and the state derivative evolution
x_values = traj
xdot_values = trajDot[:,:-1]

# Generate the trajectory corresponding to random input sequence
rand_init_traj_vec = np.zeros((n_data_max + 1, initial_state.size), dtype=np.float64)
rand_init_traj_der_vec = np.zeros((n_data_max, initial_state.size), dtype=np.float64)
rand_init_input_vec = np.zeros((n_data_max, rand_control.shape[1]), dtype=np.float64)


repeat_val = int(sampling_time/dt)
# Now redefine the state and theur derivative
for i in range(0, repeat_val*rand_control.shape[0], repeat_val):
    rand_init_input_vec[int(i/repeat_val),:] = u_values[:,i] 
    rand_init_traj_der_vec[int(i/repeat_val),:] = xdot_values[:,i]
    rand_init_traj_vec[int(i/repeat_val),:] = x_values[:,i]
rand_init_traj_vec[-1,:] = x_values[:,repeat_val*rand_control.shape[0]]

In [ ]:
# # Preview the trajectory
# plt.style.use('dark_background')

# plt.figure()
# plt.plot(tVal, xdot_values[0,:], "red", label='$\dot{p}_x$')
# plt.plot(tMeas, rand_init_traj_der_vec[:,0], 'b*', label='$\mathscr{T}_{'+ str(tMeas.shape[0])+'}$')
# plt.xlabel('Time (s)')
# plt.ylabel('$\dot{p}_x$')
# plt.legend(loc='best')
# plt.grid(True)
# plt.tight_layout()
# plt.show()

# plt.figure()
# plt.plot(tVal, xdot_values[1,:], "red", label='$\dot{p}_y$')
# plt.plot(tMeas, rand_init_traj_der_vec[:,1], 'b*', label='$\mathscr{T}_{'+ str(tMeas.shape[0])+'}$')
# plt.xlabel('Time (s)')
# plt.ylabel('$\dot{p}_y$')
# plt.legend(loc='best')
# plt.grid(True)
# plt.tight_layout()
# plt.show()

# plt.figure()
# plt.plot(tVal, xdot_values[2,:], "red", label='$\dot{\theta}$')
# plt.plot(tMeas, rand_init_traj_der_vec[:,2], 'b*', label='$\mathscr{T}_{'+ str(tMeas.shape[0])+'}$')
# plt.xlabel('Time (s)')
# plt.ylabel('$\dot{\theta}$')
# plt.legend(loc='best')
# plt.grid(True)
# plt.tight_layout()
# plt.show()

# plt.figure()
# plt.plot(x_values[0,:-1], x_values[1,:-1], 'red', label='$\mathrm{Unknown \ trajectory}$')
# plt.plot(rand_init_traj_vec[:-1,0], rand_init_traj_vec[:-1,1], 'b*', label='$\mathscr{T}_{'+ str(tMeas.shape[0])+'}$')    
# plt.xlabel('$p_x$')
# plt.ylabel('$p_y$')
# plt.legend(loc='best')
# plt.grid(True)
# plt.tight_layout()
# plt.show()

# plt.figure()
# plt.plot(tVal, x_values[2,:-1], "red", label='$\dot{\theta}$')
# plt.plot(tMeas, rand_init_traj_vec[:-1,2], 'b*', label='$\mathscr{T}_{'+ str(tMeas.shape[0])+'}$')
# plt.xlabel('Time (s)')
# plt.ylabel('$\dot{\theta}$')
# plt.legend(loc='best')
# plt.grid(True)
# plt.tight_layout()
# plt.show()

In [ ]:
# Build the over-approximations of f and G
Lf = np.array([0.01, 0.01, 0.01], dtype=realN) # Lipschitz constant of f
LG = np.array([[1.1,0],[1.1,0],[0,0.01]], dtype=realN)

nDepG = Dict(*depTypeG)
nDepG[(0,0)] = np.array([0,1],dtype=np.int64)
nDepG[(1,0)] = np.array([0,1],dtype=np.int64)

@jit(nopython=True)
def knownGfun(x_lb, x_ub):
    res_lb = np.zeros((3,2),dtype=realN)
    res_ub = np.zeros((3,2),dtype=realN)
    return res_lb, res_ub

overApprox = initOverApprox(Lf, LG,  Lfknown=None, LGknown=None, nvDepF=depTypeF,
        nvDepG=nDepG, bf=depTypebf , bG =depTypebG , bGf = depTypeGradF,
        bGG=depTypeGradG, xTraj=rand_init_traj_vec.T, xDotTraj=rand_init_traj_der_vec.T, 
        uTraj=rand_init_input_vec.T, useGronwall=True, verbose=False, Gknown=knownGfun)

In [ ]:
# Compute over-approximation at each point in x_values
xdot_lb = np.zeros((x_values.shape[0],nPoint))
xdot_ub = np.zeros((x_values.shape[0],nPoint))
for i in range(nPoint):
    f_lb, f_ub = fover(overApprox, x_values[:,i], x_values[:,i])
    G_lb, G_ub = Gover(overApprox, x_values[:,i], x_values[:,i], knownG=knownGfun)
    xdot_lb[:,i], xdot_ub[:,i] = add_i(f_lb, f_ub, *mul_Ms_i(G_lb, G_ub, u_values[:,i]))

In [ ]:
# Plot the over-approximation of the reachable set
plt.style.use('dark_background')

# First plot the state evolution
plt.figure()
plt.plot(tVal, xdot_values[0,:], "red", linewidth = 1, label='$f(x) + G(x) u$')
plt.plot(tMeas, rand_init_traj_der_vec[:,0], 'b*', label='$\mathscr{T}_{'+ str(tMeas.shape[0])+'}$')
plt.fill_between(tVal, xdot_lb[0,:], xdot_ub[0,:], alpha=0.9, facecolor='tab:green', label='$\mathrm{f}(x) + \mathrm{G}(x) u$')
plt.xlabel('Time (s)')
plt.ylabel('$\dot{p}_x$')
plt.legend(loc='best')
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure()
plt.plot(tVal, xdot_values[1,:], "red", linewidth = 1, label='$f(x) + G(x) u$')
plt.plot(tMeas, rand_init_traj_der_vec[:,1], 'b*', label='$\mathscr{T}_{'+ str(tMeas.shape[0])+'}$')
plt.fill_between(tVal, xdot_lb[1,:], xdot_ub[1,:], alpha=0.9, facecolor='tab:green', label='$\mathrm{f}(x) + \mathrm{G}(x) u$')
plt.xlabel('Time (s)')
plt.ylabel('$\dot{p}_y$')
plt.legend(loc='best')
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure()
plt.plot(tVal, xdot_values[2,:], "red", linewidth = 1, label='$f(x) + G(x) u$')
plt.plot(tMeas, rand_init_traj_der_vec[:,2], 'b*', label='$\mathscr{T}_{'+ str(tMeas.shape[0])+'}$')
plt.fill_between(tVal, xdot_lb[2,:], xdot_ub[2,:], alpha=0.9, facecolor='tab:green', label='$\mathrm{f}(x) + \mathrm{G}(x) u$')
plt.xlabel('Time (s)')
plt.ylabel('$\dot{\theta}$')
plt.legend(loc='best')
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure()
plt.plot(tVal, u_values[0,:], "red", linewidth = 1, label='$f(x) + G(x) u$')
plt.plot(tVal, u_values[1,:], "red", linewidth = 1)
plt.plot(tMeas, rand_init_input_vec[:,0], 'b*', label='$\mathscr{T}_{'+ str(tMeas.shape[0])+'}$')
plt.plot(tMeas, rand_init_input_vec[:,1], 'b*')
plt.xlabel('Time (s)')
plt.ylabel('$\dot{\theta}$')
plt.legend(loc='best')
plt.grid(True)
plt.tight_layout()
plt.show()